In [1]:
from pathlib import Path
import mgmGrowth.utils.flair2nrrd as flair2nrrd

root   = Path("/home/mariopasc/Python/Datasets/Meningiomas/men")
outdir = Path("/home/mariopasc/Python/Datasets/Meningiomas/Meningioma_Adquisition/RM")

df = flair2nrrd.batch_convert_flair(root, outdir)
df.head()


WARNING | Invalid value for VR UI: '1.2.840.113704.7.01.164.66376,1640104.15'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
/home/mariopasc/Python/misc/miniconda3/envs/mgmGrowth/lib/python3.10/site-packages/pydicom/valuerep.py:440: UserWarning: Invalid value for VR UI: '1.2.840.113704.7.01.164.66376,1640104.15'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
  warn_and_log(msg)
WARNING | Invalid value for VR UI: '1.2.840.113704.7.01.164.66376,1640104.15'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
WARNING | Invalid value for VR UI: '1.2.840.113704.7.01.164.66376,1640104.15'. Please see <https://dicom.nema.org/medical/dicom/current/output/html/part05.html#table_6.2-1> for allowed values for each VR.
WARNING | Invalid value for VR UI: '1.2.840.11370

,PatientID,Written,Slices,VoxelSpacing,DirectionDet,OutFile,Copied,Reason
0,P1,True,20.0,"(0.8984375, 0.8984374999999999, 2.5263158702795)",1.0,/home/mariopasc/Python/Datasets/Meningiomas/Me...,NaN,NaN
1,P12,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
2,P14,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
3,P15,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
4,P16,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing


In [2]:
df

,PatientID,Written,Slices,VoxelSpacing,DirectionDet,OutFile,Copied,Reason
0,P1,True,20.0,"(0.8984375, 0.8984374999999999, 2.5263158702795)",1.0,/home/mariopasc/Python/Datasets/Meningiomas/Me...,NaN,NaN
1,P12,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
2,P14,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
3,P15,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
4,P16,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
5,P19,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
6,P21,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
7,P22,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
8,P24,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
9,P26,True,NaN,NaN,NaN,/home/mariopasc/Python/Datasets/Meningiomas/Me...,True,copied_existing
